# Fourier regression

The baseline was quite unsatisfactory. You can do better than this.

Many methods for long-term forecasting aim at identifying and reconstructing the trend and seasonal structure of the series.
When sesonal components are important, as in this case, a common approach is [Fourier regression](https://otexts.com/fpp3/useful-predictors.html#fourier-series).

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

There are many academic papers and book chapters about the application of Fourier regression for time series forecasting. Here, a simple method will provide a good case study to introduce SageMaker algorithms.
    
The interested reader may refer to [A. Incremona, Machine Learning methods for long and short term energy demand forecasting](https://iris.unipv.it/retrieve/handle/11571/1436355/470615/Incremona_PhD_Thesis.pdf), [A. Incremona, G. De Nicolao, Spectral Characterization of the Multi-Seasonal Component of the Italian Electric Load: A LASSO-FFT Approach](https://ieeexplore.ieee.org/abstract/document/8734852), and [A. Guerini, G. De Nicolao, Long-term electric load forecasting: A torus-based approach](https://ieeexplore.ieee.org/abstract/document/7330957).
</div>

Here, you opt for a simple unregularized linear model:
$$
y_t = \mu + \kappa t + \sum_{i=1}^n\alpha_i\sin(2\pi\frac{i}{7}t) + \beta_i\cos(2\pi\frac{i}{7}t) + \sum_{j=1}^m\gamma_i\sin(2\pi\frac{j}{365.25}t) + \delta_j\sin(2\pi\frac{j}{365.25}t) + \tau_t
$$
where:
- the terms $\mu + \kappa t$ model a linear trend
- $\tau_t$ is a dummy variable, which takes value 1 when date $t$ is a holiday
- the sums $\sum_{i=1}^n\alpha_i\sin(2\pi\frac{i}{7}t) + \beta_i\cos(2\pi\frac{i}{7}t) + \sum_{j=1}^m\gamma_i\sin(2\pi\frac{j}{365.25}t)$  model the weekly and yearly periodicity

The parameters of the model are $\mu$, $\kappa$, $\{\alpha_i\}$, $\{\beta_i\}$, $\{\gamma_j\}$, $\{\delta_j\}$, which can all be trained by least squares. The hyperparameters are $n$ and $m$, which can be chosen by cross-validation.

Moreover, in such models it is common to log-transform the target variable, in order to stabilize the variance of the series. Therefore, another candidate model is  
$$
\log y_t = \mu + \kappa t + \sum_{i=1}^n\alpha_i\sin(2\pi\frac{i}{7}t) + \beta_i\cos(2\pi\frac{i}{7}t) + \sum_{j=1}^m\gamma_i\sin(2\pi\frac{j}{365.25}t) + \delta_j\sin(2\pi\frac{j}{365.25}t) + \tau_t
$$

# Setup
Unfortunately, you soon discover that the default packages installed in SageMaker Studio are quite outdated. 
Therefore, after some Googling, you resort to upgrade `sagemaker-experiments`, `s3fs`, and `pandas`.

Moreover, you install `holidays` to ease feature engineering.

The warnings and errors raised by package installation are not a concern for this workshop.

In [ ]:
# Add sagemaker experiments
!pip install sagemaker-experiments --upgrade

# Required for feature engineering
!pip install holidays

# To read data from S3
! pip install pandas s3fs --upgrade

Please, restart the kernel if this is the first time you run this notebook.

This is necessary to ensure that we can actually import the libraries we've just installed in the previous cells.

In [ ]:
# 'ml.m5.xlarge' is included in the AWS Free Tier
INSTANCE_TYPE = 'ml.m5.xlarge'

In [ ]:
import datetime
import os
import time
from pathlib import Path

import boto3
import holidays
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
from smexperiments.experiment import Experiment
from smexperiments.tracker import Tracker
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [ ]:
# Configuring the default size for matplotlib plots
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 6)

# Raw data gathering
The data processing pipeline created by Matteo and Gabriele deposits the final dataset in a conventional location on S3.
In order to retrieve the data to crunch, you first load the S3 object. 

With `pandas`, the integration is immediate: S3 URI are resolved as if they were file paths.

You also create some objects that will be useful throughout the notebook.

In [ ]:
boto_session = boto3.Session()
sagemaker_session = sagemaker.Session()
sagemaker_client = boto_session.client("sagemaker")
sagemaker_bucket = sagemaker_session.default_bucket()
main_prefix = "amld22-workshop-sagemaker"

raw_data_s3_path = "s3://public-workshop/normalized_data/processed/2006_2022_data.parquet"
raw_df = pd.read_parquet(raw_data_s3_path)
resampled_df = raw_df.resample('D').sum()

You are at the beginning the January 2020. So, you can only see the data until the end of 2019.

In [ ]:
NOW = '2019-12-31 23:59'
TRAIN_END = '2017-12-31 23:59'

load_df = resampled_df[:NOW].copy()
load_df.head()

# Feature preparation
To perform a Fourier regression, you need proper inputs. Therefore, you build a new dataset.

You add the trend and holiday dummy, then you build the Fourier harmonics.

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

In the real world, a good practice would be to choose the number of harmonics with data-driven methods: cross-validation is one, LASSO is another. If the latter is chosen, information criteria or cross-validation is needed to pick the most effective value of the regularizazion parameter. Here, we do not show how the hyperparameters were chosen.
</div>

In [ ]:
weekly_harmonics = 5
yearly_harmonics = 20


def create_dataset_for_regression(dates: pd.DatetimeIndex, weekly_harmonics: int,
                                  yearly_harmonics: int) -> pd.DataFrame:
    date_origin = pd.to_datetime('2000-01-01')
    holiday_cal = holidays.Italy()
    regression_df = pd.DataFrame(index=dates)
    regression_df['trend'] = (dates - date_origin).days
    regression_df['holiday'] = dates.to_series().apply(lambda s: s in holiday_cal).astype(int)
    for i in range(1, weekly_harmonics + 1):
        regression_df[f'sin_7_{i}'] = np.sin(2 * np.pi * i / 7 * regression_df['trend'])
        regression_df[f'cos_7_{i}'] = np.cos(2 * np.pi * i / 7 * regression_df['trend'])
    for i in range(1, yearly_harmonics + 1):
        regression_df[f'sin_365_{i}'] = np.sin(2 * np.pi * i / 365.25 * regression_df['trend'])
        regression_df[f'cos_365_{i}'] = np.cos(2 * np.pi * i / 365.25 * regression_df['trend'])
    return regression_df


regression_data_df = load_df.join(create_dataset_for_regression(load_df.index, weekly_harmonics, yearly_harmonics))
regression_train_df, regression_test_df = regression_data_df[:TRAIN_END], regression_data_df[TRAIN_END:]
regression_train_df.head()

## Upload on S3
SageMaker train jobs retrieve data from S3: you thus need to upload the train and the test set.

In [ ]:
s3_train_path = f's3://{sagemaker_bucket}/{main_prefix}/data/modelling/fourier-regression/train.parquet'
s3_test_path = f's3://{sagemaker_bucket}/{main_prefix}/data/modelling/fourier-regression/test.parquet'

regression_train_df.to_parquet(s3_train_path)
regression_test_df.to_parquet(s3_test_path)

# Feature store setup

You do not want to recompute all the feature every time you want to train and, most notably, make predictions.

<div class="alert alert-block alert-info">
<b>Note.</b> 

Actually, our features are very easy and quick to compute. So, it would not be inefficient to just create the proper dataset at inference time. But let's pretend this is not the case.
</div>

Therefore, you resort to the AWS SageMaker Feature Store. You compute the features once, up to a future as far as you think necessary, and you save the result in the feature store. At inference time, you can easily retrieve the records from the feature store, although this comes with some limitations - for instance, you will not be able to predict more than 100 samples at the same time.

Feel free to check out more about the feature store on the [AWS documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html).

In [ ]:
def wait_for_feature_group_creation_complete(feature_group: FeatureGroup) -> None:
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"Feature Group {feature_group.name} successfully created.")

In [ ]:
from botocore.exceptions import ClientError

date_feature_name = 'date'
event_time_feature_name = 'event_time'

feature_dates = pd.date_range('2000-01-01', '2029-12-31', freq='D')
feature_df = create_dataset_for_regression(feature_dates, weekly_harmonics, yearly_harmonics)
feature_df[date_feature_name] = pd.Series(feature_df.index.strftime('%Y-%m-%d'), dtype='string', index=feature_df.index)
feature_df[event_time_feature_name] = pd.Series(round(time.time()), dtype='float', index=feature_df.index)

feature_group = FeatureGroup(name='load-forecasting', sagemaker_session=sagemaker_session)

try:
    feature_group.load_feature_definitions(data_frame=feature_df.reset_index(drop=True))
    feature_group.create(
        s3_uri=f"s3://{sagemaker_bucket}/{main_prefix}/feature-store/{feature_group.name}",
        record_identifier_name="date",
        event_time_feature_name="event_time",
        role_arn=sagemaker.get_execution_role(),
        enable_online_store=True,
    )

    wait_for_feature_group_creation_complete(feature_group)
    feature_group.ingest(data_frame=feature_df, max_workers=3, wait=True)
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceInUse':
        print(f'Feature group {feature_group.name} already exists. Skipping creation...')
    else:
        raise e

featurestore_client = boto_session.client(service_name='sagemaker-featurestore-runtime')
featurestore_client.get_record(FeatureGroupName=feature_group.name, RecordIdentifierValueAsString='2018-01-01')

# SKLearn training

You then turn to the training step.

We will use the script `fourier_regression_train.py`.

The sklearn estimator requires an external script: when the `fit()` method of the estimator is called, the script is injected into an AWS-managed container, and the python script gets executed.

The hyperparameters of the estimator are passed to the script as command-line parameters, while the train and test dataset are injected into the container, in locations defined by conventional environment variables (`SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST`).

The trained model should be dumped to a third conventional location, available in the environment variable `SM_MODEL_DIR`, so that it can later be retrieved for inference.

More information about training sklearn models on the [AWS documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/sklearn.html).

# Experiment setup
Log transformation or no log transformation?

Why not trying both!

In order to try out different model configurations and compare the results, you adopt the model registry. For each trial you can log input, parameters, performance metrics, trained models, and many other data. 

Using an experiment registry and proper code and data versioning systems is the best way to ensure reproducibility and accountability of results.

Experiments are organized hierarchically in trials and components. Automatic charts are created for metrics, even though they are best suited for Deep Learning models (more on this later). 

You will use the results of the experiments to inspect the goodness of models, and you will leverage the SDK to automatically choose the best one to deploy.

More on the experiment registry on the [AWS documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html).

In [ ]:
def get_or_create_experiment(experiment_name: str, experiment_description: str) -> Experiment:
    experiment = None
    for exp in Experiment.list():
        if exp.experiment_name == experiment_name:
            return Experiment.load(experiment_name=experiment_name)

    if experiment is None:
        return Experiment.create(experiment_name=experiment_name, description=experiment_description)


fourier_experiment = get_or_create_experiment(
    experiment_name="load-forecasting-fourier-regression",
    experiment_description="Fourier regression for Italian load forecasting"
)

with Tracker.create(display_name='Preprocessing') as tracker:
    tracker.log_parameters({
        "weekly_harmonics": weekly_harmonics,
        "yearly_harmonics": yearly_harmonics,
    })
    preprocessing_trial_component = tracker.trial_component

# SKLearn model fit
Everything is set and ready. You can run the training job.

The job runs on ad-hoc managed virtual machines on AWS. You think about running the first trials in local mode, but you realize it works only on SageMaker Notebook, not on SageMaker Studio. What is the point, then? What can possibly go wrong?

You run two trials, both with and without the logarithmic transformation. Inspecting the logs, you note the metrics: they are extracted using suitable regex and reported SageMaker Experiments.

In [ ]:
for log_transform_target in [0, 1]:
    fourier_trial = Trial.create(
        trial_name=f"linear-regression-{datetime.datetime.utcnow().strftime('%Y%m%d-%H%M%S')}",
        experiment_name=fourier_experiment.experiment_name,
        sagemaker_boto_client=sagemaker_client,
    )

    fourier_trial.add_trial_component(preprocessing_trial_component)

    sklearn_estimator = SKLearn(
        entry_point='fourier_regression.py',
        framework_version="0.23-1",
        instance_type=INSTANCE_TYPE,
        role=sagemaker.get_execution_role(),
        sagemaker_session=sagemaker_session,
        hyperparameters={"log_transform_target": log_transform_target},
        metric_definitions=[
            {"Name": "train:mape", "Regex": "Train MAPE: (.*?);"},
            {"Name": "train:mae", "Regex": "Train MAE: (.*?);"},
            {"Name": "train:rmse", "Regex": "Train RMSE: (.*?);"},
            {"Name": "test:mape", "Regex": "Test MAPE: (.*?);"},
            {"Name": "test:mae", "Regex": "Test MAE: (.*?);"},
            {"Name": "test:rmse", "Regex": "Test RMSE: (.*?);"},
        ],
        enable_sagemaker_metrics=True,
    )

    sklearn_estimator.fit(
        inputs={
            "train": s3_train_path,
            "test": s3_test_path
        },
        experiment_config={
            "ExperimentName": fourier_experiment.experiment_name,
            "TrialName": fourier_trial.trial_name,
            "TrialComponentDisplayName": "Training",
        }
    )

# Experiment review
When the job ends, you inspect the experiment results. In particular, you care about the train and test Mean Absolute Percentage Error (MAPE).

The model with log transformation is only marginally better. You decide to promote it anyway and go on with the deployment of the serving endpoint.

In [ ]:
trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=sagemaker_session,
    experiment_name=fourier_experiment.experiment_name,
    sort_by="metrics.test:mape.max",
    sort_order="Ascending",
    metric_names=["train:mape", "test:mape"],
    parameter_names=["log_transform_target"],
    search_expression={
        "Filters": [
            {
                "Name": "DisplayName",
                "Operator": "Equals",
                "Value": "Training",
            }
        ]
    }
)

trial_component_analytics.dataframe()[['TrialComponentName', 'train:mape - Avg', 'test:mape - Avg', 'log_transform_target']]

## Best model selection
Using the SageMaker Experiments SDK you find the model with the lowest MAPE, and you create an SKLearn model for the deployment.

In [ ]:
# Pulling best based on sort in the analytics/dataframe so first is best....
best_trial_component_name = trial_component_analytics.dataframe().iloc[0]["TrialComponentName"]
best_trial_component = TrialComponent.load(best_trial_component_name)

best_sklearn_estimator = SKLearnModel(
    model_data=best_trial_component.output_artifacts["SageMaker.ModelArtifact"].value,
    role=sagemaker.get_execution_role(),
    entry_point='fourier_regression.py',
    env={"log_transform_target": str(int(best_trial_component.parameters["log_transform_target"]))},
    # parameters are saved as float by default, but the training script required int
    sagemaker_session=sagemaker_session,
    framework_version="0.23-1"
)

# Deployment
Using the facility of AWS SageMaker, you deploy the model to a managed endpoint.

On inference, the functions `model_fn` and `input_fn` of the `fourier_regression.py` module will be called and will allow the trained model to be fed with new data.
Due to limitations on the online Feature Store, only 100 samples can be predicted on each call of the API.

In [ ]:
sklearn_predictor = best_sklearn_estimator.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    serializer=sagemaker.serializers.JSONSerializer()
)

# Prediction
Finally, you use the deployed model to perform some predictions. 

You use the sklearn estimator client, but your colleagues can call the API using any REST client. 

The integration of the feature store makes the API agnostic to the features: only a list of dates is needed.

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / y_pred)

In [ ]:
test_actual_series = regression_test_df.Load.iloc[:100]

request_data = {'dates': regression_test_df.index[:100].strftime('%Y-%m-%d').tolist()}
test_predictions = sklearn_predictor.predict(request_data)
test_prediction_series = pd.Series(test_predictions, index=regression_test_df.index[:100])

fourier_mape = mean_absolute_percentage_error(test_actual_series, test_prediction_series)

plt.title(f"Fourier regression | MAPE: {100 * fourier_mape:.2f} %")
plt.plot(regression_test_df.Load.iloc[:100], label='actual')
plt.plot(test_prediction_series, label='prediction')
plt.grid(0.4)
plt.legend()
plt.show()

# A very tough year
Time flies: we are at the end of March 2020, and the world has changed.

Probably the least damaging effect of the pandemic, your models do not work anymore. You backtest your last long-term model on the period from January to March 2020, and the results are depressing. The MAPE skyrockets to unacceptable levels.

No long-term approach can account for the disruption caused by lockdowns: it is time to change approach. If you want to be accurate in a fast-changing environment, you have to resort to short-term models.

However, a worrying idea refuses to leave your mind. If a sudden, unexpected event like an epidemic happens, it is reasonable to assume all the models of human behaviours will be impacted. But if changes happen slowly, how can we capure them? How can we make sure that our models are always reliable and effective?

You walk away from your PC. Next time you will meet your good friends and fellow Data Scientists Marta and Gabriele, you will have an interesting topic to discuss.

In [ ]:
covid_dates = pd.date_range('2020-01-06', '2020-03-31', freq='D')
covid_predictions = sklearn_predictor.predict({'dates': covid_dates.strftime('%Y-%m-%d').tolist()})
covid_prediction_series = pd.Series(covid_predictions, index=covid_dates)
covid_actual_series = resampled_df.loc[covid_dates, :].Load

covid_mape = mean_absolute_percentage_error(covid_actual_series, covid_prediction_series)

plt.title(f"Fourier regression | Covid MAPE: {100 * covid_mape:.2f} %")
plt.plot(covid_actual_series, label='actual')
plt.plot(covid_prediction_series, label='prediction')
plt.legend()
plt.grid(0.4)
plt.show()

In [ ]:
covid_prediction_df = pd.DataFrame({'actual': covid_actual_series, 'predicted': covid_prediction_series})
rolling_mape_df = pd.DataFrame(
    {'rolling_mape': map(lambda w: mean_absolute_percentage_error(w["actual"], w["predicted"]),
                         covid_prediction_df.rolling(7, method="table"))},
    index=covid_prediction_df.index
)
plt.plot(rolling_mape_df.rolling_mape)
plt.title('Rolling MAPE (7-day window)')
plt.grid(0.4)
plt.show()

# Cleanup
If you’re ready to be done with this notebook, please run the cells below with `CLEANUP = True`. 

This will remove the model, hosted endpoint, and all the experiments you created to avoid any charges from a stray instance being left on.

In [ ]:
CLEANUP = True

In [ ]:
if CLEANUP:
    sklearn_predictor.delete_model()
    sklearn_predictor.delete_endpoint()
    feature_group.delete()

In [ ]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                tc.delete()
            except:
                continue
            time.sleep(.5)  # to prevent throttling
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()


if CLEANUP:
    for e in Experiment.list():
        print(f"Deleting {e.experiment_name}...")
        experiment_to_cleanup = Experiment.load(experiment_name=e.experiment_name)
        cleanup_sme_sdk(experiment_to_cleanup)
        print(f"Deleted {e.experiment_name}")